# Step 3: Meta-Model Stacking (The Final Polish)

This is the final step of the pipeline. We take the Out-of-Fold (OOF) probabilities from our 3 architectures (ResNet-18, ResNet-34, EfficientNet-B0) and use them as features to train a meta-classifier.

In [ ]:
import os, json, numpy as np, pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

DATA_DIR = '/kaggle/input/the-last-frequency'
WORKING_DIR = '/kaggle/working'

with open(f'{DATA_DIR}/label_map.json') as f: 
    label_map = {int(k): v for k, v in json.load(f).items()}

def load_stacking_data():
    r18_oof = np.load(f'{WORKING_DIR}/resnet18_oof_probs.npy')
    r34_oof = np.load(f'{WORKING_DIR}/resnet34_oof_probs.npy')
    eff_oof = np.load(f'{WORKING_DIR}/efficientnet_b0_oof_probs.npy')
    r18_test = np.load(f'{WORKING_DIR}/resnet18_test_probs.npy')
    r34_test = np.load(f'{WORKING_DIR}/resnet34_test_probs.npy')
    eff_test = np.load(f'{WORKING_DIR}/efficientnet_b0_test_probs.npy')
    y_train = np.load(f'{WORKING_DIR}/expanded_train_labels.npy')
    X_train = np.concatenate([r18_oof, r34_oof, eff_oof], axis=1)
    X_test = np.concatenate([r18_test, r34_test, eff_test], axis=1)
    return X_train, y_train, X_test

X_train, y_train, X_test = load_stacking_data()
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
meta_test_preds = []
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
    X_tr, X_val = X_train[train_idx], X_train[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]
    model = XGBClassifier(n_estimators=500, max_depth=4, learning_rate=0.05, objective='multi:softprob', tree_method='hist', random_state=42)
    model.fit(X_tr, y_tr)
    val_preds = model.predict(X_val)
    print(f"Fold {fold+1} Accuracy: {accuracy_score(y_val, val_preds):.5f}")
    meta_test_preds.append(model.predict_proba(X_test))

final_indices = np.argmax(np.mean(meta_test_preds, axis=0), axis=1)
submission = pd.DataFrame({'Id': range(len(final_indices)), 'Command': [label_map[idx] for idx in final_indices]})
submission.to_csv('submission.csv', index=False)
print("Final submission saved!")